In [ ]:
%reset -f

In [163]:
import numpy as np
import math

class Leaky_ReLU:
    def forward(x):
        data = [[max(0.05*value,value) for value in row] for row in x]
        return np.array(data, dtype=float)
    def backward(x):
        data = [[1 if value>0 else 0.05 for value in row] for row in x]
        return np.array(data, dtype=float)

class ReLU:
    def forward(x):
        return np.maximum(0, x)
    def backward(x):
        return 1. * (x > 0)

class tanh:
    def forward(x):
        return np.tanh(x);
    def backward(x):
        return 1-np.tanh(x)**2;

class sigmoid:
    def forward(x):
        return 1/(1+np.exp(-x))
    def backward(x):
        fx = sigmoid.forward(x)
        return fx*(1-fx)

# loss function and its derivative
def mse(y_true, y_pred):
    return np.mean(np.power(y_pred-y_true, 2))
def mse_prime(y_true, y_pred):
    return 2*np.mean(y_pred-y_true)

# Base class
class Layer:
    def __init__(self):
        self.input = None
        self.output = None
    def forward_propagation(self, input):
        raise NotImplementedError
    def backward_propagation(self, output_error, learning_rate):
        raise NotImplementedError

class ActivationLayer(Layer):
    def __init__(self, function):
        self.activation = function.forward
        self.activation_prime = function.backward
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = self.activation(self.input)
        return self.output
    def backward_propagation(self, output_error, learning_rate):
        return self.activation_prime(self.input) * output_error

class FCLayer(Layer):
    def __init__(self, input_size, output_size):
        self.weights = np.random.rand(input_size, output_size)
        self.bias = np.random.rand(1, output_size)
    def forward_propagation(self, input_data):
        self.input = input_data
        self.output = np.dot(self.input, self.weights) + self.bias
        return self.output
    def backward_propagation(self, output_error, learning_rate):
        input_error = np.dot(output_error, self.weights.T)
        weights_error = np.dot(self.input.T, output_error)
        self.weights -= learning_rate * weights_error
        self.bias -= learning_rate * output_error
        return input_error

class Network:
    def __init__(self):
        self.layers = []
        self.loss = None
        self.loss_prime = None
    def add(self, layer):
        self.layers.append(layer)
    def use(self, loss, loss_prime):
        self.loss = loss
        self.loss_prime = loss_prime
    def predict(self, input_data):
        samples = len(input_data)
        result = []
        for i in range(samples):
            output = input_data[i]
            for layer in self.layers:
                output = layer.forward_propagation(output)
            result.append(output)
        return result
    def fit(self, x_train, y_train, epochs, learning_rate):
        x_train = np.array([[elem] for elem in x_train])
        y_train = np.array([[elem] for elem in y_train])
        samples = len(x_train)
        for i in range(epochs):
            err = 0
            for j in range(samples):
                output = x_train[j]
                for layer in self.layers:
                    output = layer.forward_propagation(output)
                err += self.loss(y_train[j], output)
                error = self.loss_prime(y_train[j], output)
                for layer in reversed(self.layers):
                    error = layer.backward_propagation(error, learning_rate)
            #err /= samples
            #if (i+1)%1000 == 0:
            #    print('i %d | err %f' % (i+1, err))
        print('epochs=%d, error=%f, learning_rate=%f' % (epochs, err, learning_rate))
        return err
        
def DisplayPurpose(y_train, out):
    for i in range(len(y_train)):
        print(y_train[i], end = '')
        print(" ->\t ", end = '')
        print(out[i])

In [238]:
x_train = np.array([[85],[88],[91],[94],[97],[100],[103],[106],[109],[112],[115]])
y_train = np.array([[28800],[14400],[7200],[3600],[1800],[900],[450],[225],[112.5],[56.25],[28.125]])

def minus_avg(x):
    data = []
    minimum = 9999999999
    maximum = -9999999999
    for row in x:
        for value in row:
            if minimum > value:
                minimum = value
            if maximum < value:
                maximum = value
    diff = maximum - minimum
    bias = diff / 2
    for row in x:
        data.append([])
        for value in row:
            #data[-1].append((value - bias)/bias)
            data[-1].append(value/diff)
    display("diff=%f, bias=%f"%(diff, bias))
    return data, diff, bias
x_train, x_diff, x_bias = minus_avg(x_train)
y_train, y_diff, y_bias = minus_avg(y_train)

net = Network()
net.add(FCLayer(len(x_train[0]), 5))
net.add(ActivationLayer(Leaky_ReLU))
net.add(FCLayer(5, len(y_train[0])))
net.add(ActivationLayer(tanh))
net.use(mse, mse_prime)
lr = 0.1
for i in range(10):
    net.fit(x_train, y_train, epochs=1000, learning_rate=lr)
    lr /= 1.5

out = net.predict(x_train)
DisplayPurpose(y_train, out)

'diff=30.000000, bias=15.000000'

'diff=28771.875000, bias=14385.937500'

epochs=1000, error=0.468277, learning_rate=0.100000
epochs=1000, error=0.123279, learning_rate=0.066667
epochs=1000, error=0.088810, learning_rate=0.044444
epochs=1000, error=0.072169, learning_rate=0.029630
epochs=1000, error=0.062160, learning_rate=0.019753
epochs=1000, error=0.055801, learning_rate=0.013169
epochs=1000, error=0.051705, learning_rate=0.008779
epochs=1000, error=0.049055, learning_rate=0.005853
epochs=1000, error=0.047329, learning_rate=0.003902
epochs=1000, error=0.046200, learning_rate=0.002601
[1.0009775171065494] ->	 [[0.84335338]]
[0.5004887585532747] ->	 [[0.62302094]]
[0.25024437927663734] ->	 [[0.22335093]]
[0.12512218963831867] ->	 [[0.08616594]]
[0.06256109481915934] ->	 [[0.07053539]]
[0.03128054740957967] ->	 [[0.05487012]]
[0.015640273704789834] ->	 [[0.0391778]]
[0.007820136852394917] ->	 [[0.02346612]]
[0.0039100684261974585] ->	 [[0.00774285]]
[0.0019550342130987292] ->	 [[-0.00798425]]
[0.0009775171065493646] ->	 [[-0.02370741]]


In [234]:
arr = np.array([[i] for i in range(85, 116)])
out = net.predict(arr/x_diff)
DisplayPurpose(arr, np.array(out)*y_diff)

[85] ->	 [[24562.60215372]]
[86] ->	 [[22447.25921888]]
[87] ->	 [[19663.08622931]]
[88] ->	 [[16332.4808861]]
[89] ->	 [[12774.82094236]]
[90] ->	 [[9406.12882888]]
[91] ->	 [[7354.01568221]]
[92] ->	 [[5812.00281266]]
[93] ->	 [[4525.01228691]]
[94] ->	 [[3479.81582079]]
[95] ->	 [[2649.65877386]]
[96] ->	 [[2001.87158358]]
[97] ->	 [[1503.34079037]]
[98] ->	 [[1131.57173162]]
[99] ->	 [[941.42793949]]
[100] ->	 [[782.33064651]]
[101] ->	 [[649.49264578]]
[102] ->	 [[538.77606465]]
[103] ->	 [[446.6331812]]
[104] ->	 [[370.04227139]]
[105] ->	 [[306.4434615]]
[106] ->	 [[253.67768817]]
[107] ->	 [[209.93052254]]
[108] ->	 [[173.68168674]]
[109] ->	 [[143.66047831]]
[110] ->	 [[118.80693475]]
[111] ->	 [[98.23834826]]
[112] ->	 [[81.22062785]]
[113] ->	 [[67.1439652]]
[114] ->	 [[55.50226408]]
[115] ->	 [[45.87582378]]
